In [144]:
import os

In [145]:
%pwd


'C:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [146]:
os.chdir('../.')

In [147]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [148]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [149]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    val_root_dir: Path
    STATUS_FILE: str
    customer_data: Path
    all_schema: dict
    customer_data: Path
    





In [150]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [151]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([self.config.artifacts_root])

        data_validation_config = DataValidationConfig(
            val_root_dir=Path(config.val_root_dir),
            STATUS_FILE=Path(config.STATUS_FILE),
            customer_data=Path(config.customer_data),
            all_schema=schema,
        )

        return data_validation_config
        
        

In [152]:
import os
import pandas as pd
from pathlib import Path
from BankProducts import  logger

In [153]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        logger.info("Data Validation started")

    def validate_all_columns(self) -> bool:
        try:
            data = pd.read_csv(self.config.customer_data)
            logger.info(f"Data loaded from {self.config.customer_data}")

            # Normalize column names to lowercase
            data.columns = data.columns.str.lower()
            schema_cols = {col.lower(): dtype for col, dtype in self.config.all_schema.items()}

            # Optional: Convert datetime columns
            if 'transactiondate' in data.columns:
                data['transactiondate'] = pd.to_datetime(data['transactiondate'])

            if data.empty:
                logger.warning("Data is empty")
                raise ValueError("Customer data is empty.")

            logger.info(f"Data shape: {data.shape}")

            data_cols = set(data.columns)
            schema_keys = set(schema_cols.keys())

            missing_in_data = schema_keys - data_cols
            extra_in_data = data_cols - schema_keys

            validation_status = not missing_in_data and not extra_in_data

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}\n")
                if missing_in_data:
                    f.write(f"Missing columns: {missing_in_data}\n")
                if extra_in_data:
                    f.write(f"Extra columns: {extra_in_data}\n")

            logger.info(f"Validation completed. Status: {validation_status}")
            return validation_status

        except Exception as e:
            logger.exception("Exception occurred during data validation")
            raise e


In [154]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    
    # Validate CSV files
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    print("Data validation completed successfully.")
    
except Exception as e:
    print(f"Validation failed: {e}")
# The code above is a complete example of how to validate the existence and content of CSV files and database tables.

[2025-06-04 21:27:49,071: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-04 21:27:49,079: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-04 21:27:49,089: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-04 21:27:49,092: INFO: common: created directory at: artifacts]
[2025-06-04 21:27:49,093: INFO: common: created directory at: artifacts]
[2025-06-04 21:27:49,095: INFO: 2874540167: Data Validation started]


[2025-06-04 21:27:49,224: INFO: 2874540167: Data loaded from artifacts\data_ingestion\customer_data.csv]
[2025-06-04 21:27:49,247: INFO: 2874540167: Data shape: (20000, 19)]
[2025-06-04 21:27:49,251: INFO: 2874540167: Validation completed. Status: True]
Data validation completed successfully.
